In [1]:
# SETUP: To import all packages 

# FOR GEODATAFRAME
import pandas as pd
import openpyxl
import geopandas as gpd
import transbigdata as tbd
import networks as nx
import osmnx as ox
import matplotlib.pyplot as plt

# FOR SCREENSHOTS
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import datetime
import time
import sys
import os

In [2]:
# EXTRACTION: To extract shapefile (.shp) files containing edges

edges_path = 'Assets//edges.shp'
gdf_edges = gpd.read_file(edges_path)
pd.set_option('display.max_colwidth', None)

from shapely.geometry import LineString
gdf_edges['startpoint'] = gdf_edges['geometry'].apply(lambda geom: (geom.xy[1][0], geom.xy[0][0]))
gdf_edges['endpoint'] = gdf_edges['geometry'].apply(lambda geom: (geom.xy[1][-1], geom.xy[0][-1]))

gdf_edges['Filename'], gdf_edges['Date'], gdf_edges['Time'] = None, None, None

print("GDF Edges")
display(gdf_edges.head())
display(gdf_edges.shape)

GDF Edges


,u,v,key,osmid,oneway,lanes,ref,name,highway,reversed,length,maxspeed,geometry,startpoint,endpoint,Filename,Date,Time
0,244084320,6262965272,10000,916400502,1,4,4,មហាវិថី សហព័ន្ធរុស្ស៊ី,trunk,False,64.194,40,"LINESTRING (104.83380 11.53919, 104.83343 11.53875)","(11.5391927, 104.8338042)","(11.538745, 104.8334322)",None,None,None
1,244084320,3963546169,10000,916400502,1,4,4,មហាវិថី សហព័ន្ធរុស្ស៊ី,trunk,False,19.182,40,"LINESTRING (104.83392 11.53933, 104.83380 11.53919)","(11.5393258, 104.8339162)","(11.5391927, 104.8338042)",None,None,None
2,244084328,3788589822,10000,657111496,1,None,4,មហាវិថី សហព័ន្ធរុស្ស៊ី,trunk,False,11.663,40,"LINESTRING (104.85559 11.56247, 104.85569 11.56251)","(11.5624684, 104.8555919)","(11.5625056, 104.855692)",None,None,None
3,244084328,6139283311,10000,657111496,1,None,4,មហាវិថី សហព័ន្ធរុស្ស៊ី,trunk,False,95.236,40,"LINESTRING (104.85479 11.56212, 104.85524 11.56233, 104.85559 11.56247)","(11.5621173, 104.8547948)","(11.5624684, 104.8555919)",None,None,None
4,253591599,10783685028,10000,1160049979,1,None,1,មហាវិថី ព្រះមុនីវង្ស (៩៣),trunk,False,80.611,40,"LINESTRING (104.92129 11.55104, 104.92140 11.55033)","(11.5510419, 104.9212892)","(11.550325, 104.9213992)",None,None,None


(520, 18)

In [3]:
def scrapper_function(df, date, depart_time, url='https://www.google.com/maps/dir///@11.5389886,104.8310353,17z/data=!4m2!4m1!3e0?entry=ttu'):     
    gdf = df.copy(deep=True)
    gdf['Date'], gdf['Time'] = date, depart_time
    driver = webdriver.Chrome()

    count = 0
    for i in range(len(gdf)):

        # Navigate to Google Maps
        if (i>0) and ((i % 15) == 0): 
            driver.quit()
            driver = webdriver.Chrome()
        driver.get(url)
        driver.maximize_window()

        # =========================
        #        ENTER ROUTE
        # =========================
        # Identify textboxes for starting and ending destination 
        input_textboxes = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "tactile-searchbox-input"))
        )
        start_destination_input, end_destination_input = None, None
        for textbox in input_textboxes:
            if "Choose starting" in textbox.get_attribute("aria-label"):
                start_destination_input = textbox
            elif "Choose destination" in textbox.get_attribute("aria-label"):
                end_destination_input = textbox

        # Set the starting and ending destinations
        startpoint, endpoint = gdf.iloc[i]['startpoint'], gdf.iloc[i]['endpoint']
        start_destination_input.send_keys(str(startpoint))
        start_destination_input.send_keys(Keys.RETURN)
        time.sleep(0.1)
        end_destination_input.send_keys(str(endpoint))
        end_destination_input.send_keys(Keys.RETURN)

        # =========================
        #   ENTER A CERTAIN TIME
        # =========================
        # Click the "Leave now" dropdown button
        leave_now_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "goog-inline-block.goog-menu-button-dropdown"))
        )
        leave_now_button.click()
        # Click the "Depart now" dropdown button
        depart_at_option = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//div[text()='Depart at']"))
        )
        depart_at_option.click()
        time.sleep(0.01)
        # Input time
        time_input = driver.find_element(By.CLASS_NAME, "LgGJQc")
        # time_input.clear()
        time_input.send_keys(Keys.CONTROL + "a")  
        time_input.send_keys(Keys.BACKSPACE)  
        time.sleep(0.5)
        time_input.send_keys(depart_time)  

        # =========================
        #   ENTER A CERTAIN DATE
        # =========================
        # Date input button
        date_input = driver.find_element(By.CLASS_NAME, "O4EvX")
        date_input.click()
        WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@class='goog-date-picker-btn goog-date-picker-previousMonth']"))
        )

        # Click the "<<" button four times to go back n_months
        current_date, specified_date = datetime.datetime.now(), datetime.datetime.strptime(date, "%d-%b-%y")
        n_months = (current_date.year - specified_date.year) * 12 + current_date.month - specified_date.month
        for _ in range(n_months):
            month_back_button = driver.find_element(By.XPATH, "//button[@class='goog-date-picker-btn goog-date-picker-previousMonth']")
            month_back_button.click()
        # After going back four months, select the date 1
        date_1 = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//td[@class='goog-date-picker-date' and text()='1']"))
        )
        date_1.click()

        # =========================
        #  SAVE & STORE SCREENSHOT
        # =========================
        my_time = datetime.datetime.now()
        yyyy, mm, dd = my_time.year, str(my_time.month).zfill(2), str(my_time.day).zfill(2) 
        hour, min, sec = str(my_time.hour).zfill(2), str(my_time.minute).zfill(2), str(my_time.second).zfill(2)
        formatted_depart_time = depart_time.replace(':', '.')
        filename = f"{date} {formatted_depart_time} - {yyyy}{mm}{dd}{hour}{min}{sec}.png"
        folderpath = f"Downloads\\{date} {formatted_depart_time}"
        filepath = f"{folderpath}\\{filename}"
        if not os.path.exists(folderpath):
            os.makedirs(folderpath)
        time.sleep(0.8)
        driver.save_screenshot(filepath)
        gdf.loc[i, 'Filename'] = filename

        count += 1
        progress = round(100 * (count) / (len(gdf)), 2)
        print(f"\rProgress: {progress} %  Screenshots taken: {count}", end='', flush=True)

    return gdf

In [4]:
times = ['8:00 AM', '8:05 AM', '8:10 AM', '8:15 AM', '8:20 AM', '8:25 AM', '8:30 AM', '8:35 AM', '8:40 AM', '8:45 AM', '8:50 AM', '8:55 AM', '9:00 AM']
the_date = "01-Sep-23"

for the_time in times: 
    print(f"Performing webscrapping for {the_date} {the_time}...")

    gdf_results = scrapper_function(gdf_edges, the_date, the_time)
    formatted_time = the_time.replace(':', '.')
    csv_filename = f"{the_date} {formatted_time}.csv"
    csv_filepath = f"Downloads\\CSV\\{csv_filename}"
    gdf_results.to_csv(csv_filepath)
    print("")
    print(f"CSV file {csv_filename} has been saved with path '{csv_filepath}'")
    print("")

Performing webscrapping for 01-Sep-23 8:00 AM...
Progress: 0.38 %  Screenshots taken: 2

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.209)
Stacktrace:
	GetHandleVerifier [0x00007FF652B41522+60802]
	(No symbol) [0x00007FF652ABAC22]
	(No symbol) [0x00007FF652977CE4]
	(No symbol) [0x00007FF65294DFDF]
	(No symbol) [0x00007FF6529F1E57]
	(No symbol) [0x00007FF652A098D1]
	(No symbol) [0x00007FF6529EA923]
	(No symbol) [0x00007FF6529B8FEC]
	(No symbol) [0x00007FF6529B9C21]
	GetHandleVerifier [0x00007FF652E441BD+3217949]
	GetHandleVerifier [0x00007FF652E86157+3488183]
	GetHandleVerifier [0x00007FF652E7F0DF+3459391]
	GetHandleVerifier [0x00007FF652BFB8E6+823622]
	(No symbol) [0x00007FF652AC5FBF]
	(No symbol) [0x00007FF652AC0EE4]
	(No symbol) [0x00007FF652AC1072]
	(No symbol) [0x00007FF652AB18C4]
	BaseThreadInitThunk [0x00007FFB4C55257D+29]
	RtlUserThreadStart [0x00007FFB4D8AAA48+40]
